In [1]:
from src.rde import RDE
from src.strategies import build_strategy

## Optimal tracking of fractional Brownian motion

### A. Defining the Controlled System

We begin by defining the controlled system through a function that specifies the right-hand side of the equation:

$$
dY_t = b(Y_t, U_t) \, dt + \sigma(Y_t) \, dX_t
$$

Here, `dX` represents the underlying time-augmented process, structured as follows:

```python
dX[:, 0] = dt   # Time increment
dX[:, 1] = dX_t # Stochastic process increment
```
In the tracking problem $Y$ will denote the difference between the tracking signal and thetracked process, 
therefore the system is simply given by
$$
dY_t = - U_t \, dt + \, dX_t
$$
in code this becomes

In [2]:
def rde_model(Y, U, dX):
    return dX[:, 1] - U * dX[:, 0]

### B. Defining the Strategies

Next, we define the strategies used to approximate the solutions. Here, we have several degrees of freedom. Specifically, we need to choose between:

- **Open-loop** and **closed-loop** strategies
- **Linear** and **deep** strategies
- **Log-signatures** and **standard signatures**
- **Truncation levels**

To facilitate these choices, we have implemented the helper method `build_strategy`. 

Below, we will define several types of strategies and explain their functionality.


In [10]:
# Open-loop Linear Strategy
lin_strat = build_strategy(
    N=3,          # Truncation level
    space="sig",  # Standard signature
    sig_comp="tX", # Open-loop (time signature of (t, X_t))
    nn_hidden=0   # Number of hidden layers; 0 for linear strategies
)

# Closed-loop DNN Strategy
dnn_strat = build_strategy(
    N=3,
    space="log",  # Log-signature
    sig_comp="tY", # Closed-loop (time signature of (t, Y_t))
    nn_hidden=2 
)

The arguments `N`, `space` and `sig_comp` are required. Additionally, a constraint interval can be specified passing `constraint = (a, b)`. For details on the DNN architecture parameters, please refer to the implementation in `src/strategies.py`. 
Note: A combination of open- and closed-loop strategies is possible by specifying `sig_comp = 'tXY'`.

### C. Wrapping-up a trainable model

We next integrate the RDE model with the strategy models into a single PyTorch model, enabling its use in a training pipeline. This is achieved simply by initializing an instance of the `RDE` class.

In [12]:
rde_lin = RDE(rde_model=rde_model, **lin_strat)
rde_log = RDE(rde_model=rde_model, **dnn_strat)

### D. Defining the Loss Function

Next we are going to define the loss function of the model, i.e., the minimization objective `L`. In general, this should have the following signature:

```python
loss_fn(time, Y, U, X, **kwargs)
```

where `time` is the array of time discretization points, and `Y`, `U`, and `X` are the instance matrices of the corresponding processes. These are represented as arrays of shape `(m, n, d)`, where:
- `m` is the number of Monte Carlo samples,
- `n` is the number of time steps,
- `d` is the dimension of the process.

**Note:** Currently, the dimensions of `Y` and `U` must be `1`. Higher-dimensional control functionality will be added soon.

For evaluation purposes, the loss function should **not only return the Monte Carlo average of the losses** but also output the sample **variance of the loss**.

For the tracking problem, the loss function is given by:

$$
L(Y, U) = \frac{1}{2} \int_0^T \Big( (Y_t)^2 + \kappa (U_t)^2 \Big) d{t},
$$

where $\kappa > 0$ is the penalization parameter. We implement this cost functional as follows:

In [14]:
PENALTY = 0.1

def loss_fn(time, Y, U, X, **kwargs):
    payoff = 0.5 * (Y[:, :-1] ** 2 + PENALTY * U[:, :-1] ** 2)
    l_ = torch.mean(payoff) * (time[-1] - time[0])
    v = torch.var(torch.mean(payoff, dim=1) * (time[-1] - time[0]))
    return l_, v

### E. Training the Model

In [16]:
#tbc